## Init

In [1]:
import os
import logging
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from sqlalchemy import create_engine

from neuralforecast import NeuralForecast # type: ignore
from neuralforecast.models import TSMixerx, BiTCN, Autoformer, DeepAR, DeepNPTS, DilatedRNN, FEDformer, GRU, NHITS  # type: ignore
from neuralforecast.losses.pytorch import MAE, RMSE, HuberLoss, DistributionLoss, GMM, sCRPS  # type: ignore

In [2]:
load_dotenv()  # take environment variables from .env.

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

alchemyEngine = create_engine(
    db_url,
    pool_recycle=3600,
)

In [3]:
# query = """
# with cte as (
# SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
# FROM index_daily_em_view
# where symbol = '399673'
# order by date desc
# limit 1200
# ) select * from cte order by date
# """
query = """
with cte as (
SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
FROM index_daily_em_view 
where symbol = '399673'
and change_rate is not null
) select * from cte order by date
"""

raw_df = pd.read_sql(query, alchemyEngine, parse_dates=["date"])

In [4]:
df = raw_df.rename(columns={"date":"ds", "change_rate":"y"})
df.insert(0, "unique_id", "399673")

In [5]:
df.dtypes

unique_id                     object
ds                    datetime64[ns]
open                         float64
close                        float64
high                         float64
low                          float64
volume                       float64
amount                       float64
open_preclose_rate           float64
high_preclose_rate           float64
low_preclose_rate            float64
vol_change_rate              float64
amt_change_rate              float64
y                            float64
dtype: object

In [6]:
len(df)

2499

In [126]:
df.tail()

,unique_id,ds,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,y
2494,399673,2024-09-13,1513.36,1499.00,1519.81,1499.00,8896076.0,3.033779e+10,0.01190,0.43815,-0.93710,-4.06046,5.58422,-0.93710
2495,399673,2024-09-18,1496.44,1500.40,1507.33,1490.76,6867638.0,2.025240e+10,-0.17078,0.55570,-0.54970,-22.80149,-33.24363,0.09340
2496,399673,2024-09-19,1509.46,1511.24,1528.74,1485.94,11939896.0,3.250389e+10,0.60384,1.88883,-0.96374,73.85739,60.49397,0.72247
2497,399673,2024-09-20,1511.79,1502.39,1511.79,1490.51,8768317.0,2.573702e+10,0.03639,0.03639,-1.37172,-26.56287,-20.81864,-0.58561
2498,399673,2024-09-23,1504.59,1499.04,1514.83,1497.53,7963149.0,2.484293e+10,0.14643,0.82801,-0.32348,-9.18270,-3.47396,-0.22298


In [7]:
from utilsforecast.losses import mae, mse, rmse
from utilsforecast.evaluation import evaluate

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=["unique_id", "ds", "cutoff", "y"]).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.
    for cutoff in df["cutoff"].unique():
        eval_ = evaluate(df[df["cutoff"] == cutoff], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby("unique_id").mean(
        numeric_only=True
    )  # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals["best_model"] = evals.idxmin(axis=1)
    return evals

In [40]:
def train_and_evaluate(model, df_train):
    from IPython.display import display

    seed_logger = logging.getLogger("lightning_fabric.utilities.seed")
    orig_seed_log_level = seed_logger.getEffectiveLevel()
    seed_logger.setLevel(logging.FATAL)

    nf = NeuralForecast(
        models=[model],
        freq="B",
        local_scaler_type="standard",
    )

    seed_logger.setLevel(orig_seed_log_level)

    validate = True
    val_size = min(300, int(len(df_train) * 0.9)) if validate else 0
    nf.fit(df_train, val_size=val_size, use_init_models=True)

    train_losses = nf.models[0].train_trajectories
    valid_losses = nf.models[0].valid_trajectories

    print(f"train_loss: {min(train_losses, key=lambda x: x[1])[1]:.5f}")
    print(f"valid_loss: {min(valid_losses, key=lambda x: x[1])[1]:.5f}")

    forecast = nf.predict_insample()
    forecast.reset_index(inplace=True)
    evaluation_df = evaluate_cross_validation(forecast[:-val_size], mae)
    print("training MAE:")
    display(evaluation_df)

    evaluation_df = evaluate_cross_validation(forecast[-val_size:], mae)
    print("validation MAE:")
    display(evaluation_df)

In [9]:
df_train = df

In [10]:
exog = [
    "open",
    "close",
    "high",
    "low",
    "volume",
    "amount",
    "open_preclose_rate",
    "high_preclose_rate",
    "low_preclose_rate",
    "vol_change_rate",
    "amt_change_rate",
]

## TSMixerx (0.81)

In [115]:
model = TSMixerx(
    h=20,
    input_size=48,
    n_series=1,
    # stat_exog_list=["airline1"],
    # futr_exog_list=exog,
    hist_exog_list=exog,
    n_block=4,
    ff_dim=4,
    revin=True,
    # scaler_type="standard",
    max_steps=500,
    early_stop_patience_steps=-1,
    val_check_steps=5,
    learning_rate=1e-3,
    loss=HuberLoss(),
    valid_loss=HuberLoss(),
    batch_size=32,
)

Seed set to 1


In [116]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                     | Params | Mode 
-------------------------------------------------------------------------
0 | loss                | HuberLoss                | 0      | train
1 | valid_loss          | HuberLoss                | 0      | train
2 | padder              | ConstantPad1d            | 0      | train
3 | scaler              | TemporalNorm             | 0      | train
4 | norm                | ReversibleInstanceNorm1d | 2      | train
5 | temporal_projection | Linear                   | 980    | train
6 | feature_mixer_hist  | FeatureMixing            | 284    | train
7 | first_mixing        | MixingLayer              | 780    | train
8 | mixing_block        | Sequential               | 3.1 K  | train
9 | out                 | Linear                   | 5      | train
-------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TSMixerx,best_model
unique_id,,
399673,1.455645,TSMixerx


validation MAE:


,TSMixerx,best_model
unique_id,,
399673,0.813813,TSMixerx


In [123]:
exog_s = [
    # "open",
    # "close",
    # "high",
    # "low",
    # "volume",
    # "amount",
    # "open_preclose_rate",
    # "high_preclose_rate",
    # "low_preclose_rate",
    # "vol_change_rate",
    # "amt_change_rate",
]

In [124]:
model = TSMixerx(
    h=20,
    input_size=48,
    n_series=1,
    # stat_exog_list=["airline1"],
    # futr_exog_list=exog,
    hist_exog_list=exog_s,
    n_block=4,
    ff_dim=4,
    revin=True,
    # scaler_type="standard",
    max_steps=500,
    early_stop_patience_steps=-1,
    val_check_steps=5,
    learning_rate=1e-3,
    loss=HuberLoss(),
    valid_loss=HuberLoss(),
    batch_size=32,
)

Seed set to 1


In [125]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                     | Params | Mode 
-------------------------------------------------------------------------
0 | loss                | HuberLoss                | 0      | train
1 | valid_loss          | HuberLoss                | 0      | train
2 | padder              | ConstantPad1d            | 0      | train
3 | scaler              | TemporalNorm             | 0      | train
4 | norm                | ReversibleInstanceNorm1d | 2      | train
5 | temporal_projection | Linear                   | 980    | train
6 | feature_mixer_hist  | FeatureMixing            | 196    | train
7 | first_mixing        | MixingLayer              | 780    | train
8 | mixing_block        | Sequential               | 3.1 K  | train
9 | out                 | Linear                   | 5      | train
-------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TSMixerx,best_model
unique_id,,
399673,1.470753,TSMixerx


validation MAE:


,TSMixerx,best_model
unique_id,,
399673,0.836183,TSMixerx


## BiTCN (0.83)

In [89]:
model = BiTCN(
    h=20,
    input_size=48,
    # n_series=1,
    # stat_exog_list=["airline1"],
    # futr_exog_list=exog,
    hist_exog_list=exog,
    max_steps=500,
    val_check_steps=5,
    # learning_rate=1e-3,
    loss=MAE(),
    valid_loss=MAE(),
    batch_size=32,
)

Seed set to 1


In [90]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



   | Name          | Type          | Params | Mode 
---------------------------------------------------------
0  | loss          | MAE           | 0      | train
1  | valid_loss    | MAE           | 0      | train
2  | padder_train  | ConstantPad1d | 0      | train
3  | scaler        | TemporalNorm  | 0      | train
4  | lin_hist      | Linear        | 208    | train
5  | drop_hist     | Dropout       | 0      | train
6  | net_bwd       | Sequential    | 6.4 K  | train
7  | drop_temporal | Dropout       | 0      | train
8  | temporal_lin1 | Linear        | 784    | train
9  | temporal_lin2 | Linear        | 340    | train
10 | output_lin    | Linear        | 17     | train
---------------------------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params
0.031     Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,BiTCN,best_model
unique_id,,
399673,1.422493,BiTCN


validation MAE:


,BiTCN,best_model
unique_id,,
399673,0.830535,BiTCN


## AutoFormer (0.3) (F)

In [50]:
model = Autoformer(
    h=20,
    input_size=48,
    hidden_size=16,
    conv_hidden_size=32,
    n_head=2,
    loss=MAE(),
    futr_exog_list=exog,
    # scaler_type="robust",
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
)

Seed set to 1


In [51]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | decomp        | SeriesDecomp  | 0      | train
4 | enc_embedding | DataEmbedding | 224    | train
5 | dec_embedding | DataEmbedding | 224    | train
6 | encoder       | Encoder       | 4.3 K  | train
7 | decoder       | Decoder       | 3.3 K  | train
--------------------------------------------------------
8.0 K     Trainable params
0         Non-trainable params
8.0 K     Total params
0.032     Total estimated model params size (MB)
89        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,Autoformer,best_model
unique_id,,
399673,0.542566,Autoformer


validation MAE:


,Autoformer,best_model
unique_id,,
399673,0.304587,Autoformer


In [34]:
nf.models[0].metrics

{'train_loss': tensor(0.3383),
 'train_loss_step': tensor(0.3383),
 'train_loss_epoch': tensor(0.3383),
 'valid_loss': tensor(0.2603),
 'ptl/val_loss': tensor(0.2603)}

In [35]:
metrics = nf.models[0].metrics

In [36]:
float(metrics["train_loss"]), float(metrics["valid_loss"])

(0.33832964301109314, 0.2602823078632355)

In [37]:
train_losses = nf.models[0].train_trajectories
valid_losses = nf.models[0].valid_trajectories
min(train_losses, key=lambda x: x[1])[1], min(valid_losses, key=lambda x: x[1])[1]

(0.32477930188179016, 0.2602611184120178)

In [38]:
forecast = nf.predict_insample()
forecast

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                                 …

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ds,cutoff,Autoformer,y
unique_id,,,,
399673,2014-06-19,2014-06-18,-4.917983,-3.59110
399673,2014-06-20,2014-06-18,1.493997,1.08187
399673,2014-06-23,2014-06-18,5.707462,2.25177
399673,2014-06-24,2014-06-18,1.686716,0.54677
399673,2014-06-25,2014-06-18,-3.785605,-0.64595
...,...,...,...,...
399673,2024-09-12,2024-08-21,0.314536,-0.15440
399673,2024-09-13,2024-08-21,-0.474801,-0.93710
399673,2024-09-18,2024-08-21,-0.373043,0.09340


In [39]:
forecast.reset_index(inplace=True)

In [41]:
evaluation_df = evaluate_cross_validation(forecast[:-val_size], mae)
# evaluation_df.head()
evaluation_df

,Autoformer,best_model
unique_id,,
399673,0.612319,Autoformer


In [42]:
# exclude the last 200 elements of `forecast`
evaluation_df = evaluate_cross_validation(forecast[-val_size:], mae)
evaluation_df

,Autoformer,best_model
unique_id,,
399673,0.348877,Autoformer


## DeepAR (0.5) (F)

In [54]:
model = DeepAR(
    h=20,
    input_size=48,
    lstm_n_layers=3,
    trajectory_samples=100,
    loss=DistributionLoss(distribution="Normal", level=[80, 90], return_params=False),
    # loss=MAE(),
    learning_rate=0.005,
    futr_exog_list=exog,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=-1,
    enable_progress_bar=True,
)

Seed set to 1


In [55]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | valid_loss   | MQLoss           | 5      | train
2 | padder_train | ConstantPad1d    | 0      | train
3 | scaler       | TemporalNorm     | 0      | train
4 | hist_encoder | LSTM             | 336 K  | train
5 | decoder      | Decoder          | 258    | train
----------------------------------------------------------
337 K     Trainable params
10        Non-trainable params
337 K     Total params
1.349     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,DeepAR,DeepAR-median,DeepAR-lo-90,DeepAR-lo-80,DeepAR-hi-80,DeepAR-hi-90,best_model
unique_id,,,,,,,
399673,0.091031,0.097171,0.648337,0.517315,0.534627,0.668759,DeepAR


validation MAE:


,DeepAR,DeepAR-median,DeepAR-lo-90,DeepAR-lo-80,DeepAR-hi-80,DeepAR-hi-90,best_model
unique_id,,,,,,,
399673,0.532985,0.533423,0.564257,0.503241,0.873237,0.990632,DeepAR-lo-80


## DeepNPTS (0.91)

In [91]:
model = DeepNPTS(
    h=20,
    input_size=48,
    loss=MAE(),
    # futr_exog_list=exog,
    hist_exog_list=exog,
    max_steps=500,
    val_check_steps=5,
    # early_stop_patience_steps=3,
)

Seed set to 1


In [92]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | valid_loss      | MAE           | 0      | train
2 | padder_train    | ConstantPad1d | 0      | train
3 | scaler          | TemporalNorm  | 0      | train
4 | deepnptsnetwork | Sequential    | 51.3 K | train
----------------------------------------------------------
51.3 K    Trainable params
0         Non-trainable params
51.3 K    Total params
0.205     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,DeepNPTS,best_model
unique_id,,
399673,1.155806,DeepNPTS


validation MAE:


,DeepNPTS,best_model
unique_id,,
399673,0.91043,DeepNPTS


## DilatedRNN (0.83)

In [93]:
model = DilatedRNN(
    h=20,
    # input_size=24,
    loss=DistributionLoss(distribution='Normal', level=[80, 90]),
    # futr_exog_list=exog,
    hist_exog_list=exog,
    # scaler_type="robust",
    encoder_hidden_size=100,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
)

Seed set to 1


In [94]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | loss            | DistributionLoss | 5      | train
1 | padder          | ConstantPad1d    | 0      | train
2 | scaler          | TemporalNorm     | 0      | train
3 | rnn_stack       | Sequential       | 288 K  | train
4 | context_adapter | Linear           | 20.2 K | train
5 | mlp_decoder     | MLP              | 2.6 K  | train
-------------------------------------------------------------
310 K     Trainable params
5         Non-trainable params
310 K     Total params
1.243     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,DilatedRNN,DilatedRNN-median,DilatedRNN-lo-90,DilatedRNN-lo-80,DilatedRNN-hi-80,DilatedRNN-hi-90,best_model
unique_id,,,,,,,
399673,1.46559,1.466004,4.883756,3.854068,3.831717,4.865091,DilatedRNN


validation MAE:


,DilatedRNN,DilatedRNN-median,DilatedRNN-lo-90,DilatedRNN-lo-80,DilatedRNN-hi-80,DilatedRNN-hi-90,best_model
unique_id,,,,,,,
399673,0.8357,0.837277,3.899333,3.092012,2.751779,3.55637,DilatedRNN


## FEDformer (0.49) (F)

In [60]:
model = FEDformer(
    h=20,
    input_size=48,
    modes=64,
    hidden_size=64,
    conv_hidden_size=128,
    n_head=8,
    loss=MAE(),
    futr_exog_list=exog,
    # scaler_type="robust",
    learning_rate=1e-3,
    max_steps=500,
    batch_size=16,
    windows_batch_size=32,
    val_check_steps=5,
    early_stop_patience_steps=10,
)

Seed set to 1


In [61]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | decomp        | SeriesDecomp  | 0      | train
4 | enc_embedding | DataEmbedding | 896    | train
5 | dec_embedding | DataEmbedding | 896    | train
6 | encoder       | Encoder       | 78.5 K | train
7 | decoder       | Decoder       | 72.6 K | train
--------------------------------------------------------
152 K     Trainable params
0         Non-trainable params
152 K     Total params
0.611     Total estimated model params size (MB)
84        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,FEDformer,best_model
unique_id,,
399673,0.706851,FEDformer


validation MAE:


,FEDformer,best_model
unique_id,,
399673,0.493277,FEDformer


## GRU (1.4)

In [95]:
model = GRU(
    h=20,
    input_size=-1,
    loss=DistributionLoss(distribution="Normal", level=[80, 90]),
    # scaler_type="robust",
    encoder_n_layers=2,
    encoder_hidden_size=128,
    context_size=10,
    decoder_hidden_size=128,
    decoder_layers=2,
    max_steps=500,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [96]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | loss            | DistributionLoss | 5      | train
1 | padder          | ConstantPad1d    | 0      | train
2 | scaler          | TemporalNorm     | 0      | train
3 | hist_encoder    | GRU              | 153 K  | train
4 | context_adapter | Linear           | 25.8 K | train
5 | mlp_decoder     | MLP              | 1.7 K  | train
-------------------------------------------------------------
181 K     Trainable params
5         Non-trainable params
181 K     Total params
0.724     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,GRU,GRU-median,GRU-lo-90,GRU-lo-80,GRU-hi-80,GRU-hi-90,best_model
unique_id,,,,,,,
399673,1.890334,1.892182,8.47146,6.722533,5.765444,7.517992,GRU


validation MAE:


,GRU,GRU-median,GRU-lo-90,GRU-lo-80,GRU-hi-80,GRU-hi-90,best_model
unique_id,,,,,,,
399673,1.407417,1.414118,6.722407,5.322334,4.760321,6.176801,GRU


## HINT

In [12]:
from neuralforecast.models import HINT, NHITS
horizon = 20
nhits = NHITS(
    h=horizon,
    input_size=24,
    loss=GMM(n_components=10, level=[80, 90]),
    max_steps=500,
    early_stop_patience_steps=5,
    val_check_steps=5,
    # scaler_type="robust",
    learning_rate=1e-3,
    valid_loss=sCRPS(level=[80, 90]),
    futr_exog_list=exog,
    hist_exog_list=exog,
)
model = HINT(h=horizon, S=S_df.values, model=nhits, reconciliation="BottomUp")

Seed set to 1


In [20]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | GMM           | 5      | train
1 | valid_loss   | sCRPS         | 5      | train
2 | padder_train | ConstantPad1d | 0      | train
3 | scaler       | TemporalNorm  | 0      | train
4 | blocks       | ModuleList    | 3.6 M  | train
-------------------------------------------------------
3.6 M     Trainable params
10        Non-trainable params
3.6 M     Total params
14.208    Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,NHITS,NHITS-median,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,best_model
unique_id,,,,,,,
399673,0.452764,0.440586,1.078441,0.870354,0.845974,1.03145,NHITS-median


validation MAE:


,NHITS,NHITS-median,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,best_model
unique_id,,,,,,,
399673,0.514949,0.516223,0.893903,0.772258,0.718581,0.848722,NHITS


## Informer (0.37) (F)

In [64]:
from neuralforecast.models import Informer

model = Informer(
    h=20,
    input_size=48,
    hidden_size = 16,
    conv_hidden_size = 32,
    n_head = 2,
    loss=MAE(),
#  scaler_type='robust',
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    futr_exog_list=exog,
)

Seed set to 1


In [65]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 224    | train
4 | dec_embedding | DataEmbedding | 224    | train
5 | encoder       | TransEncoder  | 5.3 K  | train
6 | decoder       | TransDecoder  | 3.4 K  | train
--------------------------------------------------------
9.1 K     Trainable params
0         Non-trainable params
9.1 K     Total params
0.037     Total estimated model params size (MB)
77        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,Informer,best_model
unique_id,,
399673,0.495805,Informer


validation MAE:


,Informer,best_model
unique_id,,
399673,0.370676,Informer


## KAN (1.0)

In [97]:
from neuralforecast.models import KAN

model = KAN(
    h=20,
    input_size=38,
    loss=DistributionLoss(distribution="Normal"),
    max_steps=500,
    # scaler_type="standard",
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [98]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | layers       | ModuleList       | 2.5 M  | train
----------------------------------------------------------
2.5 M     Trainable params
5         Non-trainable params
2.5 M     Total params
10.158    Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,KAN,KAN-median,KAN-lo-90,KAN-lo-80,KAN-hi-80,KAN-hi-90,best_model
unique_id,,,,,,,
399673,0.281545,0.282374,0.800796,0.665483,0.735027,0.881514,KAN


validation MAE:


,KAN,KAN-median,KAN-lo-90,KAN-lo-80,KAN-hi-80,KAN-hi-90,best_model
unique_id,,,,,,,
399673,1.187397,1.18742,1.049831,1.056654,1.478545,1.585454,KAN-lo-90


## LSTM (1.0)

In [99]:
from neuralforecast.models import LSTM

model = LSTM(
    h=20,
    input_size=-1,
    loss=DistributionLoss(distribution="Normal", level=[80, 90]),
    # scaler_type="robust",
    encoder_n_layers=2,
    encoder_hidden_size=128,
    context_size=10,
    decoder_hidden_size=128,
    decoder_layers=2,
    max_steps=500,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [100]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | loss            | DistributionLoss | 5      | train
1 | padder          | ConstantPad1d    | 0      | train
2 | scaler          | TemporalNorm     | 0      | train
3 | hist_encoder    | LSTM             | 204 K  | train
4 | context_adapter | Linear           | 25.8 K | train
5 | mlp_decoder     | MLP              | 1.7 K  | train
-------------------------------------------------------------
232 K     Trainable params
5         Non-trainable params
232 K     Total params
0.929     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,LSTM,LSTM-median,LSTM-lo-90,LSTM-lo-80,LSTM-hi-80,LSTM-hi-90,best_model
unique_id,,,,,,,
399673,1.825305,1.82625,4.565753,3.622161,3.970702,4.940525,LSTM


validation MAE:


,LSTM,LSTM-median,LSTM-lo-90,LSTM-lo-80,LSTM-hi-80,LSTM-hi-90,best_model
unique_id,,,,,,,
399673,1.072899,1.07649,2.871083,2.258516,2.716224,3.344093,LSTM


## MLP (0.97)

In [101]:
from neuralforecast.models import MLP

model = MLP(
    h=20,
    input_size=48,
    loss=DistributionLoss(distribution='Normal', level=[80, 90]),
    # scaler_type='robust',
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [102]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | mlp          | ModuleList       | 1.6 M  | train
4 | out          | Linear           | 41.0 K | train
----------------------------------------------------------
1.7 M     Trainable params
5         Non-trainable params
1.7 M     Total params
6.726     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,MLP,MLP-median,MLP-lo-90,MLP-lo-80,MLP-hi-80,MLP-hi-90,best_model
unique_id,,,,,,,
399673,1.008219,1.008281,1.810534,1.514053,1.556206,1.870416,MLP


validation MAE:


,MLP,MLP-median,MLP-lo-90,MLP-lo-80,MLP-hi-80,MLP-hi-90,best_model
unique_id,,,,,,,
399673,0.972912,0.973646,1.25415,1.104586,1.533854,1.769622,MLP


## MLPMultivariate (0.84)

In [103]:
from neuralforecast.models import MLPMultivariate

model = MLPMultivariate(
    h=20,
    input_size=48,
    n_series=1,
    loss = MAE(),
    # scaler_type='robust',
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [104]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name   | Type          | Params | Mode 
-------------------------------------------------
0 | loss   | MAE           | 0      | train
1 | padder | ConstantPad1d | 0      | train
2 | scaler | TemporalNorm  | 0      | train
3 | mlp    | ModuleList    | 1.6 M  | train
4 | out    | Linear        | 20.5 K | train
-------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.644     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,MLPMultivariate,best_model
unique_id,,
399673,1.443913,MLPMultivariate


validation MAE:


,MLPMultivariate,best_model
unique_id,,
399673,0.848242,MLPMultivariate


## NBEATSx (0.92)

In [105]:
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MQLoss

model = NBEATSx(
    h=20,
    input_size=48,
    loss=MQLoss(level=[80, 90]),
    # scaler_type='robust',
    dropout_prob_theta=0.5,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [106]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 5      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 3.5 M  | train
4 | out          | Linear        | 2.1 K  | train
-------------------------------------------------------
3.5 M     Trainable params
2.8 K     Non-trainable params
3.5 M     Total params
13.829    Total estimated model params size (MB)
41        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,NBEATSx-median,NBEATSx-lo-90,NBEATSx-lo-80,NBEATSx-hi-80,NBEATSx-hi-90,best_model
unique_id,,,,,,
399673,1.384796,2.394228,2.000187,2.166569,2.592654,NBEATSx-median


validation MAE:


,NBEATSx-median,NBEATSx-lo-90,NBEATSx-lo-80,NBEATSx-hi-80,NBEATSx-hi-90,best_model
unique_id,,,,,,
399673,0.924255,1.424138,1.183582,1.892654,2.293179,NBEATSx-median


## NHITS (1.0)

In [107]:
from neuralforecast.models import NHITS

model = NHITS(
    h=20,
    input_size=48,
    loss=DistributionLoss(distribution="StudentT", level=[80, 90], return_params=True),
    n_freq_downsample=[2, 1, 1],
    # scaler_type='robust',
    max_steps=500,
    early_stop_patience_steps=10,
    inference_windows_batch_size=1,
    val_check_steps=5,
    learning_rate=1e-3,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [108]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | blocks       | ModuleList       | 3.1 M  | train
----------------------------------------------------------
3.1 M     Trainable params
5         Non-trainable params
3.1 M     Total params
12.424    Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,NHITS,NHITS-median,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-df,NHITS-loc,NHITS-scale,best_model
unique_id,,,,,,,,,,
399673,1.170674,1.170253,2.326868,1.822708,2.245171,2.776943,19.097923,1.169864,1.852438,NHITS-loc


validation MAE:


,NHITS,NHITS-median,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-df,NHITS-loc,NHITS-scale,best_model
unique_id,,,,,,,,,,
399673,1.001976,1.003341,1.401567,1.214474,1.349364,1.528447,15.376333,1.002531,1.104758,NHITS


## RNN (0.82)

In [109]:
from neuralforecast.models import RNN
from neuralforecast.losses.pytorch import MQLoss

model = RNN(
    h=20,
    input_size=-1,
    inference_input_size=48,
    loss=MQLoss(level=[80, 90]),
    # scaler_type="robust",
    encoder_n_layers=2,
    encoder_hidden_size=128,
    context_size=10,
    decoder_hidden_size=128,
    decoder_layers=2,
    max_steps=500,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [110]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MQLoss        | 5      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | RNN           | 51.2 K | train
4 | context_adapter | Linear        | 25.8 K | train
5 | mlp_decoder     | MLP           | 2.1 K  | train
----------------------------------------------------------
79.1 K    Trainable params
5         Non-trainable params
79.1 K    Total params
0.316     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,RNN-median,RNN-lo-90,RNN-lo-80,RNN-hi-80,RNN-hi-90,best_model
unique_id,,,,,,
399673,1.467247,5.48345,3.927219,4.160518,5.644303,RNN-median


validation MAE:


,RNN-median,RNN-lo-90,RNN-lo-80,RNN-hi-80,RNN-hi-90,best_model
unique_id,,,,,,
399673,0.824355,4.971537,3.470608,3.981521,5.482303,RNN-median


## TCN (0.83)

In [111]:
from neuralforecast.models import TCN
from neuralforecast.losses.pytorch import GMM

model = TCN(
    h=20,
    input_size=-1,
    inference_input_size=24,
    loss=GMM(n_components=7, return_params=True, level=[80, 90]),
    learning_rate=5e-4,
    kernel_size=2,
    dilations=[1, 2, 4, 8, 16],
    encoder_hidden_size=128,
    context_size=10,
    decoder_hidden_size=128,
    decoder_layers=2,
    max_steps=500,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [112]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                       | Params | Mode 
-----------------------------------------------------------------------
0 | loss            | GMM                        | 5      | train
1 | padder          | ConstantPad1d              | 0      | train
2 | scaler          | TemporalNorm               | 0      | train
3 | hist_encoder    | TemporalConvolutionEncoder | 134 K  | train
4 | context_adapter | Linear                     | 25.8 K | train
5 | mlp_decoder     | MLP                        | 3.2 K  | train
-----------------------------------------------------------------------
163 K     Trainable params
5         Non-trainable params
163 K     Total params
0.655     Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TCN,TCN-median,TCN-lo-90,TCN-lo-80,TCN-hi-80,TCN-hi-90,TCN-mu-1,TCN-std-1,TCN-mu-2,TCN-std-2,...,TCN-std-3,TCN-mu-4,TCN-std-4,TCN-mu-5,TCN-std-5,TCN-mu-6,TCN-std-6,TCN-mu-7,TCN-std-7,best_model
unique_id,,,,,,,,,,,,,,,,,,,,,
399673,1.475946,1.46927,5.205308,3.855057,4.396768,5.501355,1.59484,2.928044,2.327567,2.892324,...,2.964067,2.968152,3.692958,1.962558,2.38397,1.635559,2.005466,2.287516,2.556875,TCN-median


validation MAE:


,TCN,TCN-median,TCN-lo-90,TCN-lo-80,TCN-hi-80,TCN-hi-90,TCN-mu-1,TCN-std-1,TCN-mu-2,TCN-std-2,...,TCN-std-3,TCN-mu-4,TCN-std-4,TCN-mu-5,TCN-std-5,TCN-mu-6,TCN-std-6,TCN-mu-7,TCN-std-7,best_model
unique_id,,,,,,,,,,,,,,,,,,,,,
399673,0.852854,0.837638,3.982976,2.922151,3.636869,4.628537,1.074047,2.571046,1.611832,2.585695,...,2.711017,1.825037,3.467889,0.994688,1.997343,1.050687,1.712244,1.230357,2.40141,TCN-median


## TFT (0.83)

In [85]:
from neuralforecast.models import TFT
from neuralforecast.losses.pytorch import DistributionLoss

model = TFT(
    h=20,
    input_size=48,
    hidden_size=20,
    # loss=DistributionLoss(distribution="StudentT", level=[80, 90]),
    loss=HuberLoss(),
    learning_rate=0.005,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    # scaler_type='robust',
    windows_batch_size=None,
    enable_progress_bar=True,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [86]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | HuberLoss                | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 520    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 43.0 K | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 6.6 K  | train
6 | output_adapter          | Linear                   | 21     | train
-----------------------------------------------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.201     Total estimated mo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TFT,best_model
unique_id,,
399673,1.451976,TFT


validation MAE:


,TFT,best_model
unique_id,,
399673,0.830426,TFT


In [83]:
exog_s = [
    "open",
]
model = TFT(
    h=20,
    input_size=48,
    hidden_size=20,
    # loss=DistributionLoss(distribution="StudentT", level=[80, 90]),
    loss=HuberLoss(),
    learning_rate=0.005,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    # scaler_type='robust',
    windows_batch_size=None,
    enable_progress_bar=True,
    futr_exog_list=exog_s,
    hist_exog_list=exog_s,
)

Seed set to 1


In [84]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | HuberLoss                | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 120    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 18.1 K | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 6.6 K  | train
6 | output_adapter          | Linear                   | 21     | train
-----------------------------------------------------------------------------
24.9 K    Trainable params
0         Non-trainable params
24.9 K    Total params
0.100     Total estimated mo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TFT,best_model
unique_id,,
399673,1.441481,TFT


validation MAE:


,TFT,best_model
unique_id,,
399673,0.844393,TFT


## TiDE (1.0)

In [113]:
from neuralforecast.models import TiDE
from neuralforecast.losses.pytorch import GMM

model = TiDE(
    h=20,
    input_size=48,
    # input_size=24,
    loss=GMM(n_components=7, return_params=True, level=[80, 90]),
    max_steps=500,
    # futr_exog_list=exog,
    hist_exog_list=exog,
)

Seed set to 1


In [114]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type          | Params | Mode 
---------------------------------------------------------------
0 | loss                 | GMM           | 5      | train
1 | padder_train         | ConstantPad1d | 0      | train
2 | scaler               | TemporalNorm  | 0      | train
3 | hist_exog_projection | MLPResidual   | 8.3 K  | train
4 | dense_encoder        | Sequential    | 510 K  | train
5 | dense_decoder        | Sequential    | 920 K  | train
6 | temporal_decoder     | MLPResidual   | 6.5 K  | train
7 | global_skip          | Linear        | 13.7 K | train
---------------------------------------------------------------
1.5 M     Trainable params
5         Non-trainable params
1.5 M     Total params
5.838     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TiDE,TiDE-median,TiDE-lo-90,TiDE-lo-80,TiDE-hi-80,TiDE-hi-90,TiDE-mu-1,TiDE-std-1,TiDE-mu-2,TiDE-std-2,...,TiDE-std-3,TiDE-mu-4,TiDE-std-4,TiDE-mu-5,TiDE-std-5,TiDE-mu-6,TiDE-std-6,TiDE-mu-7,TiDE-std-7,best_model
unique_id,,,,,,,,,,,,,,,,,,,,,
399673,0.873245,0.565257,1.692992,1.211797,2.517763,4.498103,1.052521,2.334303,0.551797,1.622592,...,1.748683,1.315237,2.563862,0.601443,1.744877,2.238207,3.348994,0.525569,1.577806,TiDE-mu-7


validation MAE:


,TiDE,TiDE-median,TiDE-lo-90,TiDE-lo-80,TiDE-hi-80,TiDE-hi-90,TiDE-mu-1,TiDE-std-1,TiDE-mu-2,TiDE-std-2,...,TiDE-std-3,TiDE-mu-4,TiDE-std-4,TiDE-mu-5,TiDE-std-5,TiDE-mu-6,TiDE-std-6,TiDE-mu-7,TiDE-std-7,best_model
unique_id,,,,,,,,,,,,,,,,,,,,,
399673,1.079643,1.019356,1.480406,1.217544,2.666144,4.637601,1.433855,1.790897,1.047736,1.024954,...,1.092991,1.506781,1.96168,1.092839,1.148278,2.315489,2.947075,1.057924,1.008364,TiDE-std-7


## TimesNet (0.81) (F)

In [28]:
from neuralforecast.models import TimesNet
from neuralforecast.losses.pytorch import DistributionLoss

model = TimesNet(
    h=20,
    input_size=48,
    hidden_size = 16,
    conv_hidden_size = 32,
    loss=DistributionLoss(distribution='Normal', level=[80, 90]),
    # scaler_type='standard',
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    futr_exog_list=exog,
    # hist_exog_list=exog,
)

Seed set to 1


In [29]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | loss           | DistributionLoss | 5      | train
1 | padder_train   | ConstantPad1d    | 0      | train
2 | scaler         | TemporalNorm     | 0      | train
3 | model          | ModuleList       | 586 K  | train
4 | enc_embedding  | DataEmbedding    | 224    | train
5 | layer_norm     | LayerNorm        | 32     | train
6 | predict_linear | Linear           | 3.3 K  | train
7 | projection     | Linear           | 34     | train
------------------------------------------------------------
589 K     Trainable params
5         Non-trainable params
589 K     Total params
2.360     Total estimated model params size (MB)
52        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,TimesNet,TimesNet-median,TimesNet-lo-90,TimesNet-lo-80,TimesNet-hi-80,TimesNet-hi-90,best_model
unique_id,,,,,,,
399673,1.449127,1.449597,3.362899,2.741932,2.60229,3.213746,TimesNet


validation MAE:


,TimesNet,TimesNet-median,TimesNet-lo-90,TimesNet-lo-80,TimesNet-hi-80,TimesNet-hi-90,best_model
unique_id,,,,,,,
399673,0.811064,0.813713,2.272118,1.842059,1.797608,2.21098,TimesNet


## VanillaTransformer (0.3) (F)

In [30]:
from neuralforecast.models import VanillaTransformer
from neuralforecast.losses.pytorch import DistributionLoss

model = VanillaTransformer(
    h=20,
    input_size=48,
    hidden_size=16,
    conv_hidden_size=32,
    n_head=2,
    loss=MAE(),
    # scaler_type='robust',
    learning_rate=1e-3,
    max_steps=500,
    val_check_steps=5,
    early_stop_patience_steps=10,
    futr_exog_list=exog,
    # hist_exog_list=exog,
)

Seed set to 1


In [31]:
train_and_evaluate(model, df_train)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 224    | train
4 | dec_embedding | DataEmbedding | 224    | train
5 | encoder       | TransEncoder  | 4.5 K  | train
6 | decoder       | TransDecoder  | 3.4 K  | train
--------------------------------------------------------
8.3 K     Trainable params
0         Non-trainable params
8.3 K     Total params
0.033     Total estimated model params size (MB)
71        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

/home/kemove/git/neuralforecast/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


training MAE:


,VanillaTransformer,best_model
unique_id,,
399673,0.49577,VanillaTransformer


validation MAE:


,VanillaTransformer,best_model
unique_id,,
399673,0.308776,VanillaTransformer


## optional df_train

In [8]:
# handle univariate first, to avoid the following error
# 'DataFrame' object has no attribute 'temporal_cols'
df_train = df[["unique_id", "ds", "y", "open_preclose_rate"]]
df_train.tail()

,unique_id,ds,y,open_preclose_rate
2481,399673,2024-08-27,-0.87590,-0.56837
2482,399673,2024-08-28,-0.06461,-0.27391
2483,399673,2024-08-29,0.50911,-1.45864
2484,399673,2024-08-30,2.85494,0.22245
2485,399673,2024-09-02,-2.91054,-0.26187


# HierarchicalData

In [42]:
!pip install datasetsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.9 kB/s eta 0:00:000:10:00:20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 42.7/43.9 MB 9.3 kB/s eta 0:02:03mInstalling collected packages: appdirs, llvmlite, fs, numba, triad, adagio, fugue, datasetsforecast


In [43]:
from datasetsforecast.hierarchical import HierarchicalData

Y_df, S_df, tags = HierarchicalData.load("./data", "TourismLarge")

100%|██████████| 1.30M/1.30M [00:01<00:00, 969kiB/s] 
INFO:datasetsforecast.utils:Successfully downloaded datasets.zip, 1297274, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/hierarchical/datasets.zip


In [47]:
Y_df.shape

(126540, 3)

In [53]:
S_df.shape

(555, 304)

In [55]:
Y_df["unique_id"].unique()

['TotalAll', 'AAll', 'BAll', 'CAll', 'DAll', ..., 'GBCOth', 'GBDHol', 'GBDVis', 'GBDBus', 'GBDOth']
Length: 555
Categories (555, object): ['TotalAll', 'AAll', 'BAll', 'CAll', ..., 'GBDHol', 'GBDVis', 'GBDBus', 'GBDOth']

In [54]:
S_df.index

Index(['TotalAll', 'AAll', 'BAll', 'CAll', 'DAll', 'EAll', 'FAll', 'GAll',
       'AAAll', 'ABAll',
       ...
       'GBBBus', 'GBBOth', 'GBCHol', 'GBCVis', 'GBCBus', 'GBCOth', 'GBDHol',
       'GBDVis', 'GBDBus', 'GBDOth'],
      dtype='object', length=555)

In [56]:
S_df

,AAAHol,AAAVis,AAABus,AAAOth,AABHol,AABVis,AABBus,AABOth,ABAHol,ABAVis,...,GBBBus,GBBOth,GBCHol,GBCVis,GBCBus,GBCOth,GBDHol,GBDVis,GBDBus,GBDOth
TotalAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
AAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GBCOth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
GBDHol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
GBDVis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
GBDBus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [58]:
S_df.columns

Index(['AAAHol', 'AAAVis', 'AAABus', 'AAAOth', 'AABHol', 'AABVis', 'AABBus',
       'AABOth', 'ABAHol', 'ABAVis',
       ...
       'GBBBus', 'GBBOth', 'GBCHol', 'GBCVis', 'GBCBus', 'GBCOth', 'GBDHol',
       'GBDVis', 'GBDBus', 'GBDOth'],
      dtype='object', length=304)

In [57]:
Y_df

,unique_id,ds,y
0,TotalAll,1998-01-01,45151.071280
1,TotalAll,1998-02-01,17294.699551
2,TotalAll,1998-03-01,20725.114184
3,TotalAll,1998-04-01,25388.612353
4,TotalAll,1998-05-01,20330.035211
...,...,...,...
126535,GBDOth,2016-08-01,0.000000
126536,GBDOth,2016-09-01,0.000000
126537,GBDOth,2016-10-01,0.000000
126538,GBDOth,2016-11-01,0.000000


In [48]:
def sort_df_hier(Y_df, S_df):
    # NeuralForecast core, sorts unique_id lexicographically
    # by default, this class matches S_df and Y_hat_df order.
    Y_df.unique_id = Y_df.unique_id.astype("category")
    Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
    Y_df = Y_df.sort_values(by=["unique_id", "ds"])
    return Y_df


Y_df["ds"] = pd.to_datetime(Y_df["ds"])
Y_df = sort_df_hier(Y_df, S_df)
level = [80, 90]

In [49]:
Y_df

,unique_id,ds,y
0,TotalAll,1998-01-01,45151.071280
1,TotalAll,1998-02-01,17294.699551
2,TotalAll,1998-03-01,20725.114184
3,TotalAll,1998-04-01,25388.612353
4,TotalAll,1998-05-01,20330.035211
...,...,...,...
126535,GBDOth,2016-08-01,0.000000
126536,GBDOth,2016-09-01,0.000000
126537,GBDOth,2016-10-01,0.000000
126538,GBDOth,2016-11-01,0.000000


In [51]:
Y_df[Y_df["unique_id"] == "TotalAll"]

,unique_id,ds,y
0,TotalAll,1998-01-01,45151.071280
1,TotalAll,1998-02-01,17294.699551
2,TotalAll,1998-03-01,20725.114184
3,TotalAll,1998-04-01,25388.612353
4,TotalAll,1998-05-01,20330.035211
...,...,...,...
223,TotalAll,2016-08-01,24100.446632
224,TotalAll,2016-09-01,24800.033759
225,TotalAll,2016-10-01,30039.106985
226,TotalAll,2016-11-01,27320.918908
